In [7]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import csv
import pandas as pd

driver = webdriver.Chrome()

driver.get('https://www.redbus.in/online-booking/sikkim-nationalised-transport-snt')
sntroutedata=[]
sntp_link=[]
sntdata=[]

#rcon = driver.find_elements(By.XPATH,"//div[@class='route_link']")
#for fr in rcon:
#    print(fr.text)
sntpages =  driver.find_elements(By.CLASS_NAME,'DC_117_pageTabs')
print(len(sntpages))
for i in range(len(sntpages)):
    i = driver.find_element(By.XPATH,f"//*[@id='root']/div/div[4]/div[12]/div[{i+1}]")
    #element = driver.find_element_by_class_name('pagination-r')
    
    driver.execute_script("arguments[0].scrollIntoView();",i)
    driver.execute_script("arguments[0].click();",i) 
    rcon = driver.find_elements(By.XPATH,"//div[@class='route_link']")
    for rf in rcon:
       try:
        routename = rf.find_element(By.CSS_SELECTOR, "a.route").text
        routelinks = rf.find_element(By.CSS_SELECTOR, "a.route").get_attribute('href')
        
        #print(routename)
        #print(routelinks)
        sntroutedata.append({
            "Route_name":routename,
            "Route_link":routelinks
        })
        
        #print(routedata)
       except Exception as ex:
          
        print(ex) 
sntp_link = [i['Route_link'] for i in sntroutedata]
print(sntp_link)
print(len(sntp_link))
   

df = pd.DataFrame(sntroutedata)

df.to_csv("SNTUP_RD.csv",index=False)
for i in sntp_link:
  driver.get(i)
  time.sleep(5)
  snt(i)
time.sleep(2)
driver.quit()

2
['https://www.redbus.in/bus-tickets/gangtok-to-siliguri', 'https://www.redbus.in/bus-tickets/siliguri-to-gangtok', 'https://www.redbus.in/bus-tickets/siliguri-to-rangpo', 'https://www.redbus.in/bus-tickets/rangpo-to-siliguri', 'https://www.redbus.in/bus-tickets/siliguri-to-pelling-sikkim', 'https://www.redbus.in/bus-tickets/pelling-sikkim-to-siliguri', 'https://www.redbus.in/bus-tickets/ravangla-sikkim-to-siliguri', 'https://www.redbus.in/bus-tickets/siliguri-to-jorethang-sikkim', 'https://www.redbus.in/bus-tickets/singtham-sikkim-to-siliguri', 'https://www.redbus.in/bus-tickets/siliguri-to-singtham-sikkim', 'https://www.redbus.in/bus-tickets/siliguri-to-namchi-sikkim', 'https://www.redbus.in/bus-tickets/siliguri-to-ravangla-sikkim', 'https://www.redbus.in/bus-tickets/jorethang-sikkim-to-siliguri', 'https://www.redbus.in/bus-tickets/namchi-sikkim-to-siliguri', 'https://www.redbus.in/bus-tickets/siliguri-to-melli-sikkim']
15
Element not found within the given time
Element not found wi

In [5]:

def snt(i):
    import time
    from selenium.common.exceptions import TimeoutException
    try:

        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[class='button']")))

        element.click()
        time.sleep(5)
    except TimeoutException:
        print("Element not found within the given time")    

    time.sleep(5)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    try :
        title = driver.find_element(By.XPATH,"//div[@class='D136_heading']").text
    except:
        title = None
    bus_cont = driver.find_elements(By.XPATH,"//div[@class='clearfix bus-item']")
    time.sleep(5)
    for bus in bus_cont:
        try:
            busn = bus.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
        except:
            busn = None
        try:
            bust = bus.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
        except:
            bust = None
        try:
            dt = bus.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
        except:
            dt = None
        try:
            rt = bus.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
        except:
            rt = None
        try:
            dur = bus.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
        except:
            dur = None
        try:
            sa = bus.find_element(By.XPATH, ".//div[@class='seat-left m-top-16']").text
        except:
            sa = None
        try:
            pr = bus.find_element(By.XPATH, ".//span[@class='f-19 f-bold']").text
        except:
            pr = None
        try:
            sr = bus.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
        except:
            sr = None


        sntdata.append({

            "Route_name":title,
            "Route_link":i,        
            "Busname":busn,
            "Bustype":bust,
            "Depature_time":dt,
            "Duration":dur,
            "Reaching_time":rt,
            "Star_rating":sr,
            "Price":pr,
            "Seat_availabilty":sa
        })

        time.sleep(5) 
    df = pd.DataFrame(sntdata)
    #df.dropna(inplace=True)
    df.fillna({
        'Route_name': 'Unknown Route',
        'Route_link': 'Unknown Link',
        'Busname': 'Unknown Busname',
        'Bustype': 'Unknown Bustype',
        'Depature_time': 'Unknown Time',
        'Duration': 'Unknown Duration',
        'Reaching_time': 'Unknown Time',
        'Star_rating': 1.1,
        'Price': 0,
        'Seat_availabilty': 'No seats available',
        }, inplace=True)
    df.to_csv("SNTUP_BD.csv", index=False)



In [8]:
import pandas as pd
import mysql.connector

db = mysql.connector.connect(
    host='localhost',
    user='root',
    password='7418736408',
    database='redbus_data'
)


cursor = db.cursor()


create_table_query = """
CREATE TABLE IF NOT EXISTS bus_routes (
    id INT AUTO_INCREMENT PRIMARY KEY,
    route_name TEXT,
    route_link TEXT,
    busname TEXT,
    bustype TEXT,
    departing_time VARCHAR(255),
    duration TEXT,
    reaching_time VARCHAR(255),
    star_rating FLOAT,
    price DECIMAL(10, 2),
    seats_available VARCHAR(255)
);
"""


cursor.execute(create_table_query)
db.commit()


                
csv_file_path = 'C:/Users/ANNIE CHRISTINA G/Desktop/REDBUS/SNTUP_BD.csv'
df = pd.read_csv(csv_file_path)
#df.dropna(inplace=True)
#df['Star_rating'] = pd.to_numeric(df['Star_rating'], errors='coerce')

insert_query = """
INSERT INTO bus_routes (
    route_name, route_link, busname, bustype, departing_time, duration,
    reaching_time, star_rating, price, seats_available
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""


dat = df.to_records(index=False).tolist()


cursor.executemany(insert_query, dat)
db.commit()



